In [1]:
import sys
sys.path.append("/u/lukas.rilling/dev/")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import h5py
%matplotlib widget

from returnn_training_progress import get_epoch_data
from returnn_training_plot_nb import plot_df
%load_ext autoreload
%autoreload 2

In [3]:
globs = [
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/*/no_specaug/*/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf",
    "/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/*/no_specaug/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf",
]
lr_files = []
for g in globs:
    lr_files += glob.glob(g)

print(lr_files)
exlude = "speaker_drop"
lr_files = [l for l in lr_files if not exlude in l]
# print(lr_files)
common_prefix = os.path.commonpath(lr_files)
common_sufix = os.path.commonpath([l[::-1] for l in lr_files])[::-1]

names = []
for f in lr_files:
    names.append(f.removeprefix(common_prefix).removesuffix(common_sufix))

files = dict(zip(lr_files, names))
files, len(lr_files)

['/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/no_specaug/asr_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/basic_init/no_specaug/asr_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/basic_init/no_specaug/tts_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf', '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/no_specaug/ce_ls_0.1/phoneme_pred/train-clean/output/

({'/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/tts_pretrained/no_specaug/asr_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf': '/tts_pretrained/no_specaug/asr_target_size/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/basic_init/no_specaug/asr_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf': '/basic_init/no_specaug/asr_target_size/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training/given_alignments/raw_audio/joint_models/glowTTS_ASR_ffn_x_vector/200ep/basic_init/no_specaug/tts_target_size/ce_ls_0.1/phoneme_pred/train-clean/output/output.hdf': '/basic_init/no_specaug/tts_target_size/',
  '/u/lukas.rilling/experiments/glow_tts_asr_v2/alias/experiments/librispeech/joint_training

In [35]:
# replace_dict = {
#     "librispeech_glow_asr/pytorch/encoding_test/": "",
#     "tts_architecture/glow_tts/raw_audio/decoder_test/": "",
#     "/": " | ",
#     "_encoder_sample_test": " Sampled ",
#     "_decoder_test": " Decoder output ",
#     # "_encoding_test_blstm": "1x512 BLSTM  Mean + log(std)",
#     "_encoding_test": "",
#     "_ce": "Mean + log(std)",
#     "_mean_only": "Mean",
#     "_encoding_test_mean_only": "Mean",
#     "glowTTS": "",
#     "_maxlike_alignment": " MAS ",
#     "_simple_linear": " 1x80 linear ",
#     "_multi_layer_ffn": " 3x512 linear ",
#     "_blstm": " 1x512 BLSTM ",
#     "tts_architecture": "",
#     "enc192": "192 channels",
#     "enc768": "768 channels",
# }

In [38]:
# sorting_definition_glow_model = ["192 channels", "768 channels"]
# sorting_definition_model = [" 1x80 linear ", " 3x512 linear ", " 1x512 BLSTM "]
# sorting_definition_mean = ["mean_only", "with_sigma"]
# sorting_definition_input = ["Mean |", "Mean + log(std) |", " Sampled  |", " Sampled  MAS  |", " Decoder output  |"]
# def sorting_function(a):
#     if "mean_only" in a[1] or "with_sigma" in a[1]:
#         split_a = a[1].split(" | ")

#         sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e3

#         if "channels" in a[1]:
#             sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e2 + sorting_definition_mean.index(split_a[2]) * 1e1 + sorting_definition_input.index(split_a[3])   
#         else:
#             sorting_index_a += sorting_definition_mean.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])

#     else:
#         split_a = a[1].split(" | ")

#         sorting_index_a = sorting_definition_model.index(split_a[0]) * 1e2
#         if "channels" in a[1]:
#             sorting_index_a += sorting_definition_glow_model.index(split_a[1]) * 1e1 + sorting_definition_input.index(split_a[2])
#         else:
#             sorting_index_a += sorting_definition_input.index(split_a[1])

#     return sorting_index_a

# for k,v in new_files.items():
#     if "with_sigma" in v:
#         sorting_function([k,v])
#         break

In [39]:
# sorted_files = dict(sorted(new_files.items(), key=sorting_function))
# sorted_files.values()

dict_values([' 1x80 linear  | 768 channels | mean_only | Mean |', ' 1x80 linear  | 768 channels | mean_only |  Sampled  |', ' 1x80 linear  | 768 channels | mean_only |  Sampled  MAS  |', ' 1x80 linear  | 768 channels | mean_only |  Decoder output  |', ' 1x80 linear  | 768 channels | with_sigma | Mean |', ' 1x80 linear  | 768 channels | with_sigma | Mean + log(std) |', ' 1x80 linear  | 768 channels | with_sigma |  Sampled  |', ' 1x80 linear  | 768 channels | with_sigma |  Sampled  MAS  |', ' 1x80 linear  | 768 channels | with_sigma |  Decoder output  |', ' 3x512 linear  | 768 channels | mean_only | Mean |', ' 3x512 linear  | 768 channels | mean_only |  Sampled  |', ' 3x512 linear  | 768 channels | mean_only |  Sampled  MAS  |', ' 3x512 linear  | 768 channels | mean_only |  Decoder output  |', ' 3x512 linear  | 768 channels | with_sigma | Mean |', ' 3x512 linear  | 768 channels | with_sigma | Mean + log(std) |', ' 3x512 linear  | 768 channels | with_sigma |  Sampled  |', ' 3x512 linear  

In [8]:
print("| Model Type | Accuary | Cross Entropy |\n|============|============|============|")

for file, name in files.items():
    data = h5py.File(file)
    epoch_data = get_epoch_data(
        file.replace("phoneme_pred/train-clean/output/output.hdf", "training/work/learning_rates"), 100
    )
    if "dev_loss_ce" in epoch_data["error"]: 
        dev_loss_ce = epoch_data["error"]["dev_loss_ce"]
    else:
        dev_loss_ce = np.inf
    mean = np.array(data["inputs"]).mean()
    print(f"| {name} | {1e2*mean:.2f} | {dev_loss_ce:.3f} |")

| Model Type | Accuary | Cross Entropy |
|============|============|============|
| /tts_pretrained/no_specaug/asr_target_size/ | 31.04 | 2.872 |
| /basic_init/no_specaug/asr_target_size/ | 28.98 | 2.857 |
| /basic_init/no_specaug/tts_target_size/ | 27.46 | 2.840 |
| /tts_pretrained/no_specaug/ | 31.04 | 2.872 |
| /basic_init/no_specaug/ | 28.98 | 2.857 |
